In [2]:
import os
from os.path import join, isdir
import cv2
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers.convolutional import Convolution2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import to_categorical

Using TensorFlow backend.


In [17]:
ROOT_PATH = "../"
train_data_directory = os.path.join(ROOT_PATH, "datasets/BelgiumTSC_Training/Training/")
test_data_directory = os.path.join(ROOT_PATH, "datasets/BelgiumTSC_Testing/Testing/")

In [20]:
def load_data(data_directory):
    directories = [d for d in os.listdir(data_directory)
                  if os.path.isdir(os.path.join(data_directory, d))]
    labels = []
    images = []
    category = 0
    for d in directories:
        if (d != "Readme.txt"):
                file_names = [os.path.join(data_directory + d, f) 
                     for f in os.listdir(data_directory + d)
                     if f.endswith(".ppm")]
                
        else:
            continue
            
        for f in file_names:
            img = cv2.imread(f)
            imresize = cv2.resize(img, (200, 125))
            images.append(imresize)
            labels.append(category)
        category += 1
        
    images = np.array(images).astype("float32")
    images = images / 255.0
    
    labels = np.array(labels)
    labels = to_categorical(labels, category)
    return images, labels

In [21]:
images, labels, = load_data(train_data_directory)

In [22]:
def createCNNModel(num_classes):
    model = Sequential()
    model.add(Convolution2D(32, (3, 3), padding="same", input_shape=(125, 200, 3), activation="relu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Convolution2D(32, (3, 3), activation="relu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Convolution2D(32, (3, 3), activation="relu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(512, activation="relu"))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation="softmax"))
    
    lrate = 0.01
    decay = lrate / 30
    sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
    model.compile(loss="categorical_crossentropy", optimizer=sgd, metrics=["accuracy"])
    print(model.summary())
    return model